In [1]:
import os
import pandas as pd
import numpy as np
from utils import to_21d_vec, get_beat_vector
import re
import sys
sys.path.insert(1, './harmalysis')
from inputRN2Chord import inputRN

In [2]:
script_dir = os.getcwd()
score_rel_path = "../datasets/Sears_Corpus/sears_org_score_for_vec/op17n1i_upm2_trans_vector_pre.krn"
scorepath = os.path.join(script_dir, score_rel_path)


In [3]:
df = pd.read_csv(scorepath, sep="\t", header=None)
pd.set_option('display.max_rows', df.shape[0]+1)
#df.columns = ["harm", "root", "voice4", "voice3", "voice2", "voice1", "index", "beat", "measure", "key", "meter" ]
#df.columns = ["harm", "root", "voice4", "voice3", "voice2", "voice1", "beat", "measure", "key", "meter" ]
#df.columns = ["harm", "voice4", "voice3", "voice2", "voice1", "index", "beat", "measure", "key", "meter" ]
#df.columns = ["harm", "voice4", "voice3", "voice2", "voice1", "beat", "measure", "key", "meter" ]

#sear org
df.columns = ["voice4", "voice3", "voice2", "voice1", "harm", "beat", "measure", "key", "meter" ]

In [4]:
#process dataframe, delete bars, score representations, and "rest slice"
df = df[~df['beat'].astype(str).str.startswith(('=','.','*'))]
df

,voice4,voice3,voice2,voice1,harm,beat,measure,key,meter
7,4FF#/,4F#\,4c#/,8a#/L,I,1,=2,F#,M4/4
8,4FF#/,4F#\,4c#/,8cc#/,I,1.5,=2,F#,M4/4
9,4C#/,4B\,4e#/,8b/,V7,2,=2,F#,M4/4
10,4C#/,4B\,4e#/,8g#/J,V7,2.5,=2,F#,M4/4
11,4F#\,4A#\,4f#/,4f#/,I,3,=2,F#,M4/4
13,4r,4r,4r,8a#/,I,4.5,=2,F#,M4/4
15,4D#/,4F#\,4f#/,8b/L,IV6,1,=3,F#,M4/4
16,4D#/,4F#\,4f#/,8cc#/,IV6,1.5,=3,F#,M4/4
17,4BB/,4B\,4d#/,8dd#/,IV,2,=3,F#,M4/4
18,4BB/,4B\,4d#/,8f#/J,IV,2.5,=3,F#,M4/4


In [5]:
prev_note_list = []
for index, row in df.iterrows():
    # for this onset slice 
    this_onset_vec = []
    
    ######################################### Process notes and onsets ########################################
    voices = ["voice4","voice3","voice2","voice1"]
    cur_note_tmp = []
    for v in voices:
        this_note = ''.join(row[[v]].values)
        if ' ' in this_note: #for a part has multiple voices
            this_note_list = this_note.split(" ")
            cur_note_tmp.extend(this_note_list)
        else:
            cur_note_tmp.append(this_note)
    
    cur_note_list = []
    for n in cur_note_tmp:
        note_name = re.sub('[^a-gA-G#-]+', '', n)
        if note_name:
            cur_note_list.append(note_name)
            
    cur_onset_list = list(set(cur_note_list) - set(prev_note_list))
    prev_note_list = cur_note_list
    
    
    note_vec = to_21d_vec(cur_note_list)
    onset_vec = to_21d_vec(cur_onset_list)
    this_onset_vec.extend(note_vec)
    this_onset_vec.extend(onset_vec)
    
    ######################################### Process beat position ########################################
    beat_pos = ''.join(row[["beat"]].values)
    meter = ''.join(row[["meter"]].values)
    meter = meter.replace("M", "")
    beat_vec = get_beat_vector(beat_pos, meter)
    this_onset_vec.extend(beat_vec)
    
    ######################################### Process chord label ########################################
    harm = ''.join(row[["harm"]].values)
    harm = re.sub('[()]', '', harm) # process string
    key = ''.join(row[["key"]].values)
    key_harm = key + ":" + harm
    #print(key_harm)
    try:
        chord_label = inputRN(key_harm)["Chord label"]
    except:
        continue # pass this onset slice if RN is not recognizable
    this_onset_vec.append(str(chord_label))
    #print(this_onset_vec)
  
    #########################################################################################################


Invalid entry. Try again. C#:Gr
Invalid entry. Try again. C#:Gr
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:Gr
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:Gr
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry.

In [9]:
print(inputRN ("g:Cad64"))

{'Current key': 'G minor', 'Tonicized key': 'None', 'Intervallic construction': 'Gm3P5', 'Inversion': '2', 'Chord label': 'G minor', 'Default function': 'tonic', 'Chord pitches': "('G', 'Bb', 'D')", 'Chord pitch classes': '(7, 10, 2)'}
